## 이걸 먼저 하니 gpu가 작동함 

### https://www.kdnuggets.com/2018/02/google-colab-free-gpu-tutorial-tensorflow-keras-pytorch.html

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [4]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

gpg: keybox '/tmp/tmpatk8da0e/pubring.gpg' created
gpg: /tmp/tmpatk8da0e/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
··········
https://accounts.google.com/o/oauth2/approval/v2/approvalnativeapp?auto=false


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [2]:
!pip install -q keras
import keras
# !git clone 하면 github에 있는 library 가져올 수 잇음

Using TensorFlow backend.


In [7]:
# 작동함 옹
# !python3 drive/test/mnist_cnn.py

python3: can't open file 'drive/test/mnist_cnn.py': [Errno 2] No such file or directory


In [15]:
!ls
import os
print(os.getcwd())

data  drive  GAN.py  graphs  mnist_cnn.py  __pycache__	utils.py
/content/drive/test


In [8]:
import os
os.chdir("drive/GPU")
!ls


data  drive  GAN.py  graphs  mnist_cnn.py  __pycache__	utils.py


Found GPU at: /device:GPU:0


In [0]:
!python3 drive/GPU/GAN.py

# import 도 잘 동작함 굳굳굳 베리베리 굳굳

In [0]:
import utils

In [7]:
mnist_folder = ''
utils.download_mnist(mnist_folder)

Successfully downloaded train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz


In [0]:
!cat /proc/cpuinfo

In [10]:
!df -h

Filesystem              Size  Used Avail Use% Mounted on
overlay                  40G  4.9G   33G  14% /
tmpfs                   6.4G     0  6.4G   0% /dev
tmpfs                   6.4G     0  6.4G   0% /sys/fs/cgroup
/dev/sda1                46G  5.6G   40G  13% /etc/hosts
shm                      64M     0   64M   0% /dev/shm
tmpfs                   6.4G     0  6.4G   0% /sys/firmware
google-drive-ocamlfuse   15G   11G  4.5G  71% /content/drive


In [0]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import numpy as np
import tensorflow as tf
import time

import utils

# Define paramaters for the model
learning_rate = 0.01
batch_size = 128
n_epochs = 30
n_train = 60000
n_test = 10000
mnist_folder = ''

In [10]:
train, val, test = utils.read_mnist(mnist_folder, flatten=True)

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./data/mnist/", one_hot=True)
train = (img_train,label_train) =mnist.train.next_batch(mnist.train.num_examples)
len(train)
test = (img_test, label_test) = mnist.test.next_batch(mnist.test.num_examples)

# Step 2: Create datasets and iterator
train_data = tf.data.Dataset.from_tensor_slices(train)
train_data = train_data.shuffle(10000) # if you want to shuffle your data
train_data = train_data.batch(batch_size)

test_data = tf.data.Dataset.from_tensor_slices(test)
test_data = test_data.batch(batch_size)

iterator = tf.data.Iterator.from_structure(train_data.output_types, 
                                           train_data.output_shapes)
img, label = iterator.get_next()

train_init = iterator.make_initializer(train_data)	# initializer for train_data
test_init = iterator.make_initializer(test_data)	# initializer for train_data


Extracting ./data/mnist/train-images-idx3-ubyte.gz
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz


In [11]:
# Step 3: create weights and bias
# w is initialized to random variables with mean of 0, stddev of 0.01
# b is initialized to 0
# shape of w depends on the dimension of X and Y so that Y = tf.matmul(X, w)
# shape of b depends on Y
w = tf.get_variable(name='weights', shape=(784, 10), initializer=tf.random_normal_initializer(0, 0.01))
b = tf.get_variable(name='bias', shape=(1, 10), initializer=tf.zeros_initializer())

# Step 4: build model
# the model that returns the logits.
# this logits will be later passed through softmax layer
logits = tf.matmul(img, w) + b 

# Step 5: define loss function
# use cross entropy of softmax of logits as the loss function
entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=label, name='entropy')
loss = tf.reduce_mean(entropy, name='loss') # computes the mean over all the examples in the batch

# Step 6: define training op
# using gradient descent with learning rate of 0.01 to minimize loss
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

# Step 7: calculate accuracy with test set
preds = tf.nn.softmax(logits)
correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(label, 1))
accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [12]:


writer = tf.summary.FileWriter('./graphs/logreg', tf.get_default_graph())
with tf.Session() as sess:
   
    start_time = time.time()
    sess.run(tf.global_variables_initializer())

    # train the model n_epochs times
    for i in range(n_epochs): 	
        sess.run(train_init)	# drawing samples from train_data
        total_loss = 0
        n_batches = 0
        try:
            while True:
                _, l = sess.run([optimizer, loss])
                total_loss += l
                n_batches += 1
        except tf.errors.OutOfRangeError:
            pass
        print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))
    print('Total time: {0} seconds'.format(time.time() - start_time))

    # test the model
    sess.run(test_init)			# drawing samples from test_data
    total_correct_preds = 0
    try:
        while True:
            accuracy_batch = sess.run(accuracy)
            total_correct_preds += accuracy_batch
    except tf.errors.OutOfRangeError:
        pass

    print('Accuracy {0}'.format(total_correct_preds/n_test))
writer.close()

Average loss epoch 0: 0.3662607740870742
Average loss epoch 1: 0.29599718247042145
Average loss epoch 2: 0.2848653904747131
Average loss epoch 3: 0.279537585932155
Average loss epoch 4: 0.27476456534377364
Average loss epoch 5: 0.27470216073615605
Average loss epoch 6: 0.27079274417009463
Average loss epoch 7: 0.26827921248799147
Average loss epoch 8: 0.2657178122117076
Average loss epoch 9: 0.266206881663827
Average loss epoch 10: 0.26395270308089813
Average loss epoch 11: 0.263112930104483
Average loss epoch 12: 0.2638381868947384
Average loss epoch 13: 0.2607987284313801
Average loss epoch 14: 0.2598072331831899
Average loss epoch 15: 0.25901376852809
Average loss epoch 16: 0.2582168173478093
Average loss epoch 17: 0.25621457742743714
Average loss epoch 18: 0.2594838782105335
Average loss epoch 19: 0.2566551431147165
Average loss epoch 20: 0.2568548236302165
Average loss epoch 21: 0.25551354500443435
Average loss epoch 22: 0.2552153832344122
Average loss epoch 23: 0.2547730884233186